In [31]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, cross_validation, neighbors, manifold
from sklearn import cross_validation, metrics
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

from sklearn.neural_network import MLPClassifier as nn


In [32]:
#importing data
data_set = pd.read_csv("~/Desktop/Dataset_spine.csv",header = 0)
data_set.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Class_att,Unnamed: 13
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123,Abnormal,NaN
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102,Abnormal,NaN
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221,Abnormal,Prediction is done by using binary classificat...
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329,Abnormal,NaN
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171,Abnormal,NaN


In [33]:
#data stored per column, x is all the data mines the last column: y, since y contains the classification of the data
data_set['Class_Ord'] = pd.Categorical(data_set.Class_att).codes 

X_Y = data_set[[
'Col1',
'Col2',
'Col3',
'Col4',
'Col5',
'Col6',
'Col7',
'Col8',
'Col9',
'Col10',
'Col11',
'Col12',
'Class_Ord'
]] 

#X = preprocessing.normalize(np.array(X_Y.drop(['Class_Ord'], 1)),norm='l1')
X = np.array(X_Y.drop(['Class_Ord'], 1))

X_best_features = np.array(X_Y.drop(['Class_Ord','Col7',
'Col8',
'Col9',
'Col10',
'Col11',
'Col12',
'Col4',
],1))

Y = np.array(data_set['Class_Ord'])

X_no_outlier = np.delete(X_best_features,(115),axis=0)
Y_no_outlier = np.delete(Y,(115),axis=0)


In [34]:
#To use the different data input; best_features, no_oulier, change the X in test_split.
#When using no_oulier, also Y has to be Y_no_outlier due to the change in input size.
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size = 0.2)


mlp = nn(hidden_layer_sizes=(12,12),
                    activation='logistic', solver='adam',
                    alpha=0.0001, batch_size='auto',
                    learning_rate='constant',
                    learning_rate_init=0.001,
                    power_t=0.5, max_iter=10000,
                    shuffle=True,
                    random_state=None,
                    tol=0.0001,
                    verbose=False,
                    warm_start=False,
                    momentum=0.9,
                    nesterovs_momentum=True,
                    early_stopping=False,
                    validation_fraction=0.1,
                    beta_1=0.9,
                    beta_2=0.999,
                    epsilon=1e-08)


n=1
amount_of_iterations = 50
predict_accuracy_list = []
predict_class1 = []

while n<=amount_of_iterations:
    mlp.fit(X_train, y_train)
    predict_accuracy = mlp.score(X_test, y_test)
    predict_accuracy_list = np.append(predict_accuracy_list, predict_accuracy)
    predict_class1 = mlp.predict(X_test)
    n = n+1
    
print "average_prediction:",np.average(predict_accuracy_list)

predict_class = mlp.predict(X_test)

f1 = metrics.f1_score(y_test, predict_class)
print "f1-score:",f1


#Confusion matrix
cf_matrix = metrics.confusion_matrix(y_test, predict_class1)
print "confusion matrix:"
print cf_matrix

average_prediction: 0.837419354839
f1-score: 0.782608695652
confusion matrix:
[[34  8]
 [ 2 18]]


In [18]:
#running this will give a graph in an external window, this needs to be closed again before anything else can be run!!
######################################################################################################################

#performing manifold on the data
mds = manifold.MDS()
co = mds.fit(X_no_outlier, y=Y_no_outlier, init=None)
co_t = mds.fit_transform(X, y=Y)
maxindex = co_t.argmax()
matrix_co_t = np.matrix(co_t)
plt.plot(matrix_co_t[:,0], matrix_co_t[:,1], 'x')
plt.show()